### Flemin's function of second order model in python

In [3]:
import numpy as np
from scipy.stats import multivariate_normal, norm

def compute_meta_conf(xp, a, sigma_act, sigma_conf, rho):

    dhat = np.array([-1, 1])
    mu_x_xp_dhat = np.zeros((2, len(xp)))
    var_x_xp_dhat = np.zeros(len(xp))
    rho_vec = np.full(len(xp), rho)
    sigA_vec = np.full(len(xp), sigma_act)
    sigP_vec = np.full(len(xp), sigma_conf)
    
    Tol = 10e-4

    for dhati in range(2):
        dhat_vec = np.full(len(xp), dhat[dhati])
        
        mu_x_xp_dhat[dhati, :] = dhat_vec + (sigA_vec / sigP_vec) * rho_vec * (xp - dhat_vec)
        var_x_xp_dhat = (1 - rho_vec**2) * sigA_vec**2
        
        if a == 1:
            p_a_dhat_xp = 1 - norm.cdf(0, mu_x_xp_dhat[dhati, :], np.sqrt(var_x_xp_dhat))
        else:
            p_a_dhat_xp = norm.cdf(0, mu_x_xp_dhat[dhati, :], np.sqrt(var_x_xp_dhat))
        
        lik_d = norm.pdf(xp, dhat_vec, sigP_vec)
        
        if dhati == 0:
            p_a_dhat_xp_full = p_a_dhat_xp
            lik_d_full = lik_d
        else:
            p_a_dhat_xp_full = np.vstack((p_a_dhat_xp_full, p_a_dhat_xp))
            lik_d_full = np.vstack((lik_d_full, lik_d))
    
    # manage probability
    p_a_dhat_xp_full = np.clip(p_a_dhat_xp_full, Tol, None)
    lik_d_full = np.clip(lik_d_full, Tol, None)
    
    lik_d_full = lik_d_full / np.sum(lik_d_full, axis=0, keepdims=True)
    p_dhat_xp_a = p_a_dhat_xp_full * lik_d_full
    p_dhat_xp_a = p_dhat_xp_a / np.sum(p_dhat_xp_a, axis=0, keepdims=True)
    
    # Conf = p(a=d)
    if a == 1:
        conf = p_dhat_xp_a[1, :]
    else:
        conf = p_dhat_xp_a[0, :]
    
    return conf


### Simulation of 10 trials

In [8]:
theta = [0, 0.032, 0.064, 0.128, 0.256, 0.512, 1]
sigmaAct = 1
sigmaConf = 1
rho = 0.5
bigSigma = np.array([[sigmaAct**2, rho * sigmaAct * sigmaConf], [rho * sigmaAct * sigmaConf, sigmaConf**2]])

N = 10  # N trials

xa = np.empty((len(theta), N))
xp = np.empty((len(theta), N))
d = np.empty((len(theta), N))
a = np.empty((len(theta), N))
secondOrder_mean_cor = np.empty((len(theta), N))

for s in range(len(theta)):
    for i in range(N):
        d[s, i] = 1 if np.random.rand() > 0.5 else -1
        
        r = multivariate_normal.rvs(mean=[d[s, i] * theta[s], d[s, i] * theta[s]], cov=bigSigma)
        xa[s, i] = r[0]
        xp[s, i] = r[1]
        
        if xa[s, i] > 0:
            a[s, i] = 1
            flip_a = 1
        else:
            a[s, i] = -1
            flip_a = 0
        
        secondOrder_mean_cor[s, i] = compute_meta_conf(np.array([xp[s, i]]), flip_a, sigmaAct, sigmaConf, rho)[0]

print("(d):", d)
print("(a):", a)
print("Second Order Confidences:", secondOrder_mean_cor)

(d): [[-1. -1. -1. -1.  1. -1.  1.  1. -1. -1.]
 [ 1.  1.  1.  1.  1. -1.  1. -1. -1. -1.]
 [ 1.  1. -1.  1.  1. -1. -1. -1.  1.  1.]
 [-1. -1. -1.  1. -1.  1. -1.  1.  1.  1.]
 [-1. -1. -1.  1. -1.  1. -1.  1.  1. -1.]
 [-1. -1.  1. -1. -1. -1.  1. -1. -1.  1.]
 [-1. -1. -1. -1. -1. -1.  1. -1. -1. -1.]]
(a): [[ 1.  1.  1. -1. -1.  1.  1.  1. -1. -1.]
 [ 1. -1. -1.  1. -1. -1.  1. -1.  1.  1.]
 [-1. -1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1. -1.  1.  1. -1. -1.  1.  1.  1. -1.]
 [ 1.  1. -1. -1.  1.  1. -1. -1.  1. -1.]
 [-1. -1.  1.  1. -1.  1.  1. -1. -1.  1.]
 [-1. -1. -1. -1. -1. -1.  1.  1.  1. -1.]]
Second Order Confidences: [[0.75697275 0.93033164 0.91190662 0.86495219 0.51733263 0.9667263
  0.97133247 0.50774461 0.91376084 0.71269183]
 [0.56334765 0.94587022 0.83282217 0.77840549 0.69181068 0.98870921
  0.94473636 0.93095566 0.71094468 0.93037608]
 [0.41986993 0.97846901 0.6975973  0.98836575 0.97386285 0.94279524
  0.84708618 0.86240679 0.3086348  0.59248011]
 [0.49344233 0.9